In [ ]:
!pip install openai
!pip install torch==1.13.1
!pip install torchdata==0.5.1
!pip install transformers==4.27.2
!pip install datasets==2.11.0

In [ ]:
# import os
# from openai import OpenAI

# openai.api_key = ""

# messages = []

# labeled_dataset = [{"text" : "I do not like this", "label": "negative"}, {"text": "I like this", "label": "positive"}]
# # Chat GPT labeled examples
# few_shot_message = ""
# # Mention the Task
# few_shot_message = "Task: Sentiment Classification \n"
# # Mention the classes
# few_shot_message += "Classes: positive, negative \n"
# # Add context
# few_shot_message += "Context: We want to classify sentiment of hotel reviews \n"
# #Add labeled examples
# few_shot_message += "Labeled Examples: \n"
# for labeled_data in labeled_dataset:
#     few_shot_message += "Text: " + labeled_data["text"] + "\n Label: " + labeled_data["label"] + "\n"

# few_shot_message += "Now we will ask you the sentiment classification for a  few unlabeled examples. Please use labeled examples as context."
# messages.append({"role": "user", "content": few_shot_message})


# client = OpenAI(
#     api_key="",
# )

# chat = client.chat.completions.create(
#             model="gpt-3.5-turbo", messages=messages
#         )

# unlabeled_dataset = ["I hate this", "I love this"]
# for data in unlabeled_dataset:
#     # Add the text to classfy
#     message = "Text: " + data + ", "
#     # Add the prompt
#     message += "Prompt: Classify the given text into one of the following sentiment categories: positive, negative"
#     messages.append({"role": "user", "content": message})
#     chat = client.chat.completions.create(
#             model="gpt-3.5-turbo", messages=messages
#         )
#     reply = chat.choices[0].message.content
#     print(f"ChatGPT: {reply}")
#     messages.append({"role": "assistant", "content": reply})

In [14]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Load Dataset
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

# Load Model  & Tokenizer
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Show example data
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(i)
  print('Input')
  print(dataset['test'][index]['dialogue'])
  print('Summary')
  print(dataset['test'][index]['summary'])

  0%|          | 0/3 [00:00<?, ?it/s]

0
Input
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
Summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
1
Input
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful har

In [15]:
# select an example
example_index = [40, 200]

for index in example_index:
  # get the dialogue
  dialogue = dataset['test'][index]['dialogue']

  # get the human summary
  summary = dataset['test'][index]['summary']

  # Configurations (We will use Greedy Approach)
  generation_config = GenerationConfig(max_new_tokens=50)

  # Encode input
  inputs_encoded = tokenizer(dialogue, return_tensors='pt')

  # Model Output
  model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

  # Decode the output
  zero_output = tokenizer.decode(model_output, skip_special_tokens=True)

  print(zero_output)

Person1: It's ten to nine.
#Person1#: I'm thinking of upgrading my computer.


In [16]:
def make_prompt(example_indices, example_index_to_sum):
  "write correct prompt from given sentences. Add the latest prompt for predicted sentence"
  prompt = ""

  for index in example_indices:

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # prompt includes the dialogue
    prompt += f"""
    Dialogue:

    {dialogue}

    What is going on?
    {summary}



    """

  # get the example dialog
  dialogue = dataset['test'][example_index_to_sum]['dialogue']
  prompt += f"""
    Dialogue:

    {dialogue}

    What is going on?
    """

  return prompt

In [17]:
example_indices = [40, 80, 120]
example_index_to_sum = 200
few_shot_prompt = make_prompt(example_indices, example_index_to_sum)

# Encode input THE ONLY DIFFERENCE IS HERE
inputs_encoded = tokenizer(few_shot_prompt, return_tensors='pt')

# Model Output
model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

# Decode the output
few_output = tokenizer.decode(model_output, skip_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


In [19]:
print("Input: ", dialogue)
print(dash_line)
print( "Human summary: ", summary)
print(dash_line)
print("Zero-Shot Output: ", zero_output)
print(dash_line)
print("Few-Shot Output: ", few_output)

Input:  #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
---------------------------------------------------------------------------------------------------
Human summary:  #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
--------------------